### XGBoost

In [ ]:
import pandas as pd

# Read in the data
folder_path = 'C:\\Users\\aoife\Documents\\Project\\filtered_data\\'

save_path = 'C:\\Users\\aoife\Documents\\Project\\'

# Read in merged file
df = pd.read_csv(folder_path + 'concatenated_data.csv', usecols=['participantId', 'bodyMass_kg', 'height_m', 'bmi', 'mean_hr/s', 'hr_duration_(s)', 'max_hr', 'min_hr', 'totalSteps', 'stepsTotalDuration', 'totalDistance', 'dwrTotalDuration', 'ssq_score'])

print(df.head(5))

In [ ]:
# X - features

X = df[['bodyMass_kg','height_m','bmi','mean_hr/s','hr_duration_(s)','max_hr','min_hr','totalSteps','stepsTotalDuration','totalDistance','dwrTotalDuration']].values

# y - target
y = df['ssq_score'].values

groups = df['participantId'].values

In [ ]:
# Train test split

from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=10)


In [ ]:
import xgboost as xg

for train_index, test_index in gkf.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert train and test to DMatrix
    dtrain = xg.DMatrix(data=X_train, label=y_train)
    dtest = xg.DMatrix(data=X_test, label=y_test)


    # Set parameters
    parameters = {'booster':'gbtree', 'objective':'reg:squarederror', 'seed':42}

    model = xg.train(params=parameters, dtrain=dtrain)

    train_predictions = model.predict(dtrain)

    test_predictions = model.predict(dtest)
   

In [ ]:
# Plot predictions vs actual

import matplotlib.pyplot as plt


#plt.scatter(train_predictions, y_train, label='Train')
plt.scatter(test_predictions, y_test, label='Test')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
plt.legend
plt.show()

'''sns.scatterplot(x=predictions, y=y_test)
plt.xlabel('Predictions')
plt.ylabel('Actual')
plt.title('Predictions vs Actual')
plt.show()'''